In [1]:
from rosbags.rosbag2 import Writer
import os
import cv2
from cv_bridge import CvBridge
from evo.tools.file_interface import read_kitti_poses_file, write_bag_trajectory
from evo.core.trajectory import PoseTrajectory3D
import matplotlib.pyplot as plt
from evo.tools import plot
from evo.tools.plot import PlotMode
from scipy.spatial.transform import Rotation
import numpy as np
from builtin_interfaces.msg import Time
from sensor_msgs.msg import Image
from rclpy.serialization import serialize_message


folder = "/home/joshuabird/Desktop/00"
bridge = CvBridge()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
file_path = f"{folder}/times.txt"

poses = read_kitti_poses_file(f"{folder}/00.txt")

with open(file_path, 'r') as file:
    timestamps = file.readlines()
    timestamps = [float(timestamp.strip()) for timestamp in timestamps]

trajectory = PoseTrajectory3D(poses.positions_xyz, poses.orientations_quat_wxyz, timestamps, poses.poses_se3)
R = Rotation.from_euler('xyz', [-90, 0, 0], degrees=True).as_matrix()
T = np.hstack((np.vstack((R, [0,0,0])), [[0], [0], [0], [1]]))
trajectory.transform(T)

In [8]:
image_filenames = sorted(
    [filename for filename in os.listdir(f"{folder}/image_0") if filename.endswith(('.jpg', '.png'))]
)

topic_name = "cam0/image_raw"
msgtype = "sensor_msgs/msg/Image"


with Writer("/home/joshuabird/Desktop/00_bag") as writer:
    connection = writer.add_connection(topic_name, msgtype)

    write_bag_trajectory(writer, trajectory, "/leica/position")
    
    for i, filename in enumerate(image_filenames):
        print(filename)
        image_path = os.path.join(f"{folder}/image_0", filename)
        image = cv2.imread(image_path)
        img_msg = bridge.cv2_to_imgmsg(image, "bgr8")
        img_msg.header.stamp = Time(sec=int(timestamps[i]), nanosec=int((timestamps[i] - int(timestamps[i])) * 1e9))
        serialized_msg = serialize_message(img_msg)
        writer.write(connection, int(timestamps[i] * 1e9), serialized_msg)


000000.png
000001.png
000002.png
000003.png
000004.png
000005.png
000006.png
000007.png
000008.png
000009.png
000010.png
000011.png
000012.png
000013.png
000014.png
000015.png
000016.png
000017.png
000018.png
000019.png
000020.png
000021.png
000022.png
000023.png
000024.png
000025.png
000026.png
000027.png
000028.png
000029.png
000030.png
000031.png
000032.png
000033.png
000034.png
000035.png
000036.png
000037.png
000038.png
000039.png
000040.png
000041.png
000042.png
000043.png
000044.png
000045.png
000046.png
000047.png
000048.png
000049.png
000050.png
000051.png
000052.png
000053.png
000054.png
000055.png
000056.png
000057.png
000058.png
000059.png
000060.png
000061.png
000062.png
000063.png
000064.png
000065.png
000066.png
000067.png
000068.png
000069.png
000070.png
000071.png
000072.png
000073.png
000074.png
000075.png
000076.png
000077.png
000078.png
000079.png
000080.png
000081.png
000082.png
000083.png
000084.png
000085.png
000086.png
000087.png
000088.png
000089.png
000090.png

In [14]:
from rosbags.typesys.types import geometry_msgs__msg__PoseStamped